In [31]:
# Step 1: Imports
import pandas as pd
import matplotlib.pyplot as plt

# Step 2: Load file
df = pd.read_csv("output/squad_verbalized_confidence_fewshot_json_clean_v3.csv")  # adjust filename if needed
print("Columns:", df.columns)
df.head()


Columns: Index(['id', 'title', 'question', 'context', 'answers', 'is_impossible',
       'prompt', 'model_output', 'parsed_answer', 'parsed_confidence'],
      dtype='object')


,id,title,question,context,answers,is_impossible,prompt,model_output,parsed_answer,parsed_confidence
0,57269ef4f1498d1400e8e52d,The_Sun_(United_Kingdom),What's another thing the paper showed hostilit...,Despite its initial opposition to the closures...,['promotion of right-wing ministers to the cab...,False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""EU"", ""confidence"": 0} {""answer"": ""...",NaN,NaN
1,56dfbd5e231d4119001abd5a,Internet_service_provider,what type of technology is used to connect to ...,"ISPs provide Internet access, employing a ran...",['wireless Ethernet (wi-fi)'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""wi-fi"", ""confidence"": 100}",wi-fi,100.0
2,5730d22faca1c71400fe5af0,United_States_Air_Force,What is one of the important functions of the ...,"Offensive Counterair (OCA) is defined as ""offe...",['suppression/destruction of enemy air defense'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""suppression"", ""confidence"": 95} {""...",NaN,NaN
3,57266715dd62a815002e83be,British_Empire,When did England first establish a colony in G...,The Caribbean initially provided England's mos...,['1604'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""1604"", ""confidence"": 90}",1604,90.0
4,5706371452bb89140068996f,MP3,What would an example of lossy audio encoding be?,"When performing lossy audio encoding, such as ...",['creating an MP3 file'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""MP3"", ""confidence"": 95} # One ans...",NaN,NaN


In [35]:
df["model_output"]

0    {"answer": "EU", "confidence": 0} {"answer": "...
1               {"answer": "wi-fi", "confidence": 100}
2    {"answer": "suppression", "confidence": 95} {"...
3                 {"answer": "1604", "confidence": 90}
4    {"answer": "MP3", "confidence": 95}  # One ans...
Name: model_output, dtype: object

In [33]:
import re
import json

def extract_last_json_block(text):
    try:
        # Match all JSON objects in the text
        matches = re.findall(r'\{.*?\}', text, re.DOTALL)
        if matches:
            return json.loads(matches[-1])  # use the last one
    except Exception:
        return {"answer": None, "confidence": None}
    return {"answer": None, "confidence": None}

# Apply to DataFrame
parsed = df["model_output"].apply(extract_last_json_block)
df["parsed_answer"] = parsed.apply(lambda x: x.get("answer"))
df["parsed_confidence"] = pd.to_numeric(parsed.apply(lambda x: x.get("confidence")), errors="coerce")


In [34]:
df

,id,title,question,context,answers,is_impossible,prompt,model_output,parsed_answer,parsed_confidence
0,57269ef4f1498d1400e8e52d,The_Sun_(United_Kingdom),What's another thing the paper showed hostilit...,Despite its initial opposition to the closures...,['promotion of right-wing ministers to the cab...,False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""EU"", ""confidence"": 0} {""answer"": ""...",Social,0
1,56dfbd5e231d4119001abd5a,Internet_service_provider,what type of technology is used to connect to ...,"ISPs provide Internet access, employing a ran...",['wireless Ethernet (wi-fi)'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""wi-fi"", ""confidence"": 100}",wi-fi,100
2,5730d22faca1c71400fe5af0,United_States_Air_Force,What is one of the important functions of the ...,"Offensive Counterair (OCA) is defined as ""offe...",['suppression/destruction of enemy air defense'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""suppression"", ""confidence"": 95} {""...",suppression,95
3,57266715dd62a815002e83be,British_Empire,When did England first establish a colony in G...,The Caribbean initially provided England's mos...,['1604'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""1604"", ""confidence"": 90}",1604,90
4,5706371452bb89140068996f,MP3,What would an example of lossy audio encoding be?,"When performing lossy audio encoding, such as ...",['creating an MP3 file'],False,Example 1:\nQuestion: What is the capital of F...,"{""answer"": ""MP3"", ""confidence"": 95} # One ans...",MP3,95


In [37]:
df["model_output"][0]

'{"answer": "EU", "confidence": 0} {"answer": "Public spending cuts", "confidence": 0} {"answer": "Social security cutbacks", "confidence": 0} {"answer": "Right-wing ministers", "confidence": 0} {"answer": "Redwood", "confidence": 0} {"answer": "Deadwood", "confidence": 0} {"answer": "Thatcherite policies", "confidence": 0} {"answer": "Conservative", "confidence": 0} {"answer": "National", "confidence": 0} {"answer": "Economic", "confidence": 0} {"answer": "Labor", "confidence": 0} {"answer": "Liberal", "confidence": 0} {"answer": "Free market", "confidence": 0} {"answer": "Social", "confidence": 0} {"answer": "Right-wing", "confidence": 0} {"answer": "Free", "confidence": 0} {"answer": "Economic", "confidence": 0} {"answer": "Public", "confidence": 0} {"answer": "Social", "confidence": 0} {"answer": "'